Skeleton Code

In [46]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()
print(df.shape)
print(df.head())

<ipython-input-46-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)


(7352, 561)
        0         1         2         3         4         5         6    \
0  0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
1  0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
2  0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
3  0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
4  0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   

        7         8         9    ...       551       552       553       554  \
0 -0.983185 -0.923527 -0.934724  ... -0.074323 -0.298676 -0.710304 -0.112754   
1 -0.974914 -0.957686 -0.943068  ...  0.158075 -0.595051 -0.861499  0.053477   
2 -0.963668 -0.977469 -0.938692  ...  0.414503 -0.390748 -0.760104 -0.118559   
3 -0.982750 -0.989302 -0.938692  ...  0.404573 -0.117290 -0.482845 -0.036788   
4 -0.979672 -0.990441 -0.942469  ...  0.087753 -0.351471 -0.699205  0.123320   

        555       556       557       558       559     

<ipython-input-46-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [48]:
y.values

array([[5],
       [5],
       [5],
       ...,
       [2],
       [2],
       [2]])

In [49]:
print(df.isnull().sum())

0      0
1      0
2      0
3      0
4      0
      ..
556    0
557    0
558    0
559    0
560    0
Length: 561, dtype: int64


In [50]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, 0 to 560
dtypes: float64(561)
memory usage: 31.5 MB
None


In [51]:
print(y[0].unique())

[5 4 6 1 3 2]


In [52]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
count,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,...,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000,7352.000000
mean,0.274488,-0.017695,-0.109141,-0.605438,-0.510938,-0.604754,-0.630512,-0.526907,-0.606150,-0.468604,...,0.125293,-0.307009,-0.625294,0.008684,0.002186,0.008726,-0.005981,-0.489547,0.058593,-0.056515
std,0.070261,0.040811,0.056635,0.448734,0.502645,0.418687,0.424073,0.485942,0.414122,0.544547,...,0.250994,0.321011,0.307584,0.336787,0.448306,0.608303,0.477975,0.511807,0.297480,0.279122
min,-1.000000,-1.000000,-1.000000,-1.000000,-0.999873,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-0.995357,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.262975,-0.024863,-0.120993,-0.992754,-0.978129,-0.980233,-0.993591,-0.978162,-0.980251,-0.936219,...,-0.023692,-0.542602,-0.845573,-0.121527,-0.289549,-0.482273,-0.376341,-0.812065,-0.017885,-0.143414
50%,0.277193,-0.017219,-0.108676,-0.946196,-0.851897,-0.859365,-0.950709,-0.857328,-0.857143,-0.881637,...,0.134000,-0.343685,-0.711692,0.009509,0.008943,0.008735,-0.000368,-0.709417,0.182071,0.003181
75%,0.288461,-0.010783,-0.097794,-0.242813,-0.034231,-0.262415,-0.292680,-0.066701,-0.265671,-0.017129,...,0.289096,-0.126979,-0.503878,0.150865,0.292861,0.506187,0.359368,-0.509079,0.248353,0.107659
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,1.000000,0.478157,1.000000


In [53]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical columns:", categorical_cols)

Categorical columns: []


In [54]:

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())

In [55]:
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
sd=StandardScaler()
df[numeric_features]=sd.fit_transform(df[numeric_features])

In [56]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
count,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,7352.000000,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,...,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,7.352000e+03,7352.000000,7.352000e+03
mean,-3.131336e-16,4.783986e-17,8.698156e-17,-1.507680e-16,0.000000,2.938044e-16,3.865847e-17,-9.278034e-17,5.412186e-17,1.932924e-17,...,-9.664618e-18,6.571940e-17,2.532130e-16,3.382616e-18,-2.319508e-17,-1.932924e-17,3.479263e-17,1.005120e-16,0.000000,1.546339e-17
std,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,1.000068,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,...,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,1.000068e+00,1.000068,1.000068e+00
min,-1.814049e+01,-2.407152e+01,-1.573085e+01,-8.793362e-01,-0.972792,-9.440787e-01,-8.713436e-01,-9.736247e-01,-9.511122e-01,-9.759168e-01,...,-4.483658e+00,-2.144460e+00,-1.217541e+00,-2.925683e+00,-2.235646e+00,-1.658375e+00,-2.079788e+00,-9.974223e-01,-3.558777,-3.380417e+00
25%,-1.638693e-01,-1.756427e-01,-2.092798e-01,-8.631868e-01,-0.929530,-8.968638e-01,-8.562288e-01,-9.286823e-01,-9.034203e-01,-8.587814e-01,...,-5.936244e-01,-7.339585e-01,-7.162086e-01,-3.866541e-01,-6.507937e-01,-8.072166e-01,-7.749059e-01,-6.301983e-01,-0.257102,-3.113525e-01
50%,3.850502e-02,1.167141e-02,8.206943e-03,-7.594273e-01,-0.678376,-6.081593e-01,-7.551036e-01,-6.800064e-01,-6.061248e-01,-7.585419e-01,...,3.468984e-02,-1.142597e-01,-2.809130e-01,2.451015e-03,1.507318e-02,1.477537e-05,1.174412e-02,-4.296252e-01,0.415109,2.138846e-01
75%,1.988854e-01,1.693906e-01,2.003738e-01,8.081625e-01,0.948462,8.177048e-01,7.966913e-01,9.471052e-01,8.222278e-01,8.291412e-01,...,6.526596e-01,5.608611e-01,3.947685e-01,4.222002e-01,6.484291e-01,8.178415e-01,7.644196e-01,-3.816570e-02,0.637933,5.882181e-01
max,1.032661e+01,2.493878e+01,1.958529e+01,3.577947e+00,2.839526,3.833088e+00,3.845150e+00,3.075829e+00,3.878713e+00,2.697113e+00,...,3.272840e+00,4.039226e+00,5.144111e+00,2.943656e+00,2.225893e+00,1.627550e+00,2.096610e+00,2.910568e+00,1.410491,3.785390e+00


In [57]:
print(y)

      0
0     5
1     5
2     5
3     5
4     5
...  ..
7347  2
7348  2
7349  2
7350  2
7351  2

[7352 rows x 1 columns]


In [58]:
X_train, X_test, y_train, y_test = train_test_split(df, encoded_y, test_size=0.2, random_state=42)

In [59]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
start=time.time()
model=GaussianNB()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
end=time.time()
time_taken=end-start
print("Acuuracy : ",accuracy_score(y_test,y_pred))
print(time_taken)

Acuuracy :  0.7314751869476547
0.12538790702819824


In [64]:
n_clusters=50
kmeans=KMeans(n_clusters=n_clusters,random_state=42,n_init=10)
kmeans.fit(df.T)
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df.iloc[:,selected_features_indices]

In [61]:
selected_features.shape

(7352, 50)

In [65]:
x_train, x_test, y_train, y_test = train_test_split(selected_features, encoded_y, test_size=0.2, random_state=42)

In [66]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
start=time.time()
model=GaussianNB()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
end=time.time()
time_taken=end-start
print("Acuuracy : ",accuracy_score(y_test,y_pred))
print(time_taken)

Acuuracy :  0.8470428280081577
0.015482187271118164
